In [9]:
import pandas as pd
import problem
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer
import matplotlib.pyplot as plt
import seaborn as sns 
import create_db as cdb
import holidays
from datetime import datetime
from merge_transformer import MergeTransformer

In [10]:
X_train, y_train = problem.get_train_data('..')

In [11]:
X_train['Days_to_departure'] = (X_train['WeeksToDeparture'] * 7).round()

In [12]:
X_train.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,Days_to_departure
0,2012-06-19,ORD,DFW,12.875000,9.812647,90.0
1,2012-09-10,LAS,DEN,14.285714,9.466734,100.0
2,2012-10-05,DEN,LAX,10.863636,9.035883,76.0
3,2011-10-09,ATL,ORD,11.480000,7.990202,80.0
4,2012-02-21,DEN,SFO,11.450000,9.517159,80.0


In [13]:
X_train['DateOfDeparture'] = pd.to_datetime(X_train['DateOfDeparture'])
X_train['DateBooked'] = X_train['DateOfDeparture'] -  pd.to_timedelta(X_train['Days_to_departure'], unit='d')

In [14]:
X_train

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,Days_to_departure,DateBooked
0,2012-06-19,ORD,DFW,12.875000,9.812647,90.0,2012-03-21
1,2012-09-10,LAS,DEN,14.285714,9.466734,100.0,2012-06-02
2,2012-10-05,DEN,LAX,10.863636,9.035883,76.0,2012-07-21
3,2011-10-09,ATL,ORD,11.480000,7.990202,80.0,2011-07-21
4,2012-02-21,DEN,SFO,11.450000,9.517159,80.0,2011-12-03
...,...,...,...,...,...,...,...
8897,2011-10-02,DTW,ATL,9.263158,7.316967,65.0,2011-07-29
8898,2012-09-25,DFW,ORD,12.772727,10.641034,89.0,2012-06-28
8899,2012-01-19,SFO,LAS,11.047619,7.908705,77.0,2011-11-03
8900,2013-02-03,ORD,PHL,6.076923,4.030334,43.0,2012-12-22


# gen_df_state_feature

In [15]:
date_df = cdb.gen_date_df()

state_features_df = pd.read_csv("../data/unemployment_rate.csv", sep=';', index_col='State')
state_features_df.stack()
state_features_df = pd.DataFrame(state_features_df.stack()).reset_index(drop=False)
state_features_df.rename(columns={"State": "State", "level_1": "DateOfDeparture", 0: "UnemploymentRate"}, inplace=True)
state_features_df.loc[:, 'DateOfDeparture'] = pd.to_datetime(state_features_df.loc[:, 'DateOfDeparture'], format='%d/%m/%Y')
date_encoder = FunctionTransformer(cdb._encode_dates)
state_features_df = date_encoder.fit_transform(state_features_df)
state_features_df.rename(columns={"DateOfDeparture": "Date"}, inplace=True)

merge_transform = MergeTransformer(
    X_ext=date_df,
    cols_to_keep=['DateOfDeparture', 'year', 'month'],
    how='right',
    on=['year', 'month'],
    parse_dates=None)

state_features_df = merge_transform.fit_transform(state_features_df)
states_codes = pd.read_csv("../data/states.csv")

merge_transform = MergeTransformer(
    X_ext=states_codes, 
    filename=None,
    filepath=None, 
    cols_to_rename={'State': 'State'},
    how='left',
    on=['State'],
    parse_dates=None)

state_features_df = merge_transform.fit_transform(state_features_df)

state_features_df['DateOfDeparture'] = pd.to_datetime(state_features_df['DateOfDeparture'], format='%d/%m/%Y')
state_features_df.drop('State', axis=1, inplace=True)
state_features_df.rename({'Abbreviation': 'state'}, axis=1, inplace=True)

state_features_df['bank_holidays'] = state_features_df.apply(lambda x: x.Date in holidays.US(years = x.Date.year, state=x.state), axis=1)

In [16]:
state_features_df.head()

,Date,UnemploymentRate,year,month,day,weekday,week,n_days,DateOfDeparture,state,bank_holidays
0,2011-01-01,9.3,2011,1,1,5,52,14975,2011-01-09,AL,True
1,2011-01-01,7.7,2011,1,1,5,52,14975,2011-01-09,AK,True
2,2011-01-01,9.6,2011,1,1,5,52,14975,2011-01-09,AZ,True
3,2011-01-01,7.8,2011,1,1,5,52,14975,2011-01-09,AR,True
4,2011-01-01,12.4,2011,1,1,5,52,14975,2011-01-09,CA,True


In [26]:
def days_to_closest_holiday(date):
    
    if type(date) is pd.Timestamp:
        date = date.date()
    
    holiday_list = holidays.US(years = [2011, 2012, 2013])
    min_diff = 10e9
        
    for k, v in holiday_list.items():
        diff = abs((date - k).days)
#         print(f"Holiday = {k}, diff = {diff} days")
        if diff < min_diff:
            min_diff = diff
            
    return 1 / (min_diff + 1) ** 2

In [27]:
date_df = cdb.gen_date_df()
date_df['closest_holidays'] = date_df.apply(lambda x: days_to_closest_holiday(x.DateOfDeparture), axis=1)

In [28]:
date_df.head(50)

,DateOfDeparture,year,month,day,weekday,week,n_days,value,closest_holidays
0,2011-01-09,2011,1,9,6,1,14983,NaN,0.012346
1,2011-01-10,2011,1,10,0,2,14984,NaN,0.015625
2,2011-01-11,2011,1,11,1,2,14985,NaN,0.020408
3,2011-01-12,2011,1,12,2,2,14986,NaN,0.027778
4,2011-01-13,2011,1,13,3,2,14987,NaN,0.040000
5,2011-01-14,2011,1,14,4,2,14988,NaN,0.062500
6,2011-01-15,2011,1,15,5,2,14989,NaN,0.111111
7,2011-01-16,2011,1,16,6,2,14990,NaN,0.250000
8,2011-01-17,2011,1,17,0,3,14991,NaN,1.000000
9,2011-01-18,2011,1,18,1,3,14992,NaN,0.250000


In [ ]:
test_date = state_features_df.loc[0, 'Date']
min_diff = days_to_closest_holiday(test_date)
print(min_diff)